# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
sc = pyspark.SparkContext(appName='jupyter')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-10 16:11:19,529 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [4]:
from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

In [5]:
! hadoop fs -copyFromLocal clickstream.csv

copyFromLocal: `clickstream.csv': File exists


In [6]:
df = se.read.csv("clickstream.csv", header=True, inferSchema=True, sep='\t')

In [7]:
type(df)

pyspark.sql.dataframe.DataFrame

In [8]:
df.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- session_id: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_page: string (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
df.show()

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
|    562|       507|       event|    rabota|1695584181|
|    562|       507|       event|    rabota|1695584189|
|    562|       507|        page|      main|1695584194|
|    562|       507|       event|      main|1695584204|
|    562|       507|       event|      main|1695

In [10]:
df.registerTempTable("clicks")

### **1. SQL Solution**

* **STEP 1**:
    * Use the LAG function to create a derived table (`DERIVED TABLE 1`) containing a column (`lag_event`) with timestampes lagged by 1 position
    * Self-join the derived table with the original `clicks` table


* **STEP 2**: Derive a table (`DERIVED TABLE 2`) by extracting rows where the `event_type` column contains an error event (e.g. `wNaxLlerrorU`)


* **STEP 3**: Join the DERIVED TABLE 1 and the DERIVED TABLE 2 so that any timestamp occurring after error is tagged with `1` (1 = `error`) within both the same `user-id` and the same `session-id`


* **STEP 4**: Derive a final table (`FINAL TABLE`) by filtering out errors and rows where the current event is the same as the previous event.


* **STEP 5**: Take the FINAL TABLE to construct a route list for each `user/session` combination 


* **STEP 6**: Count the resulting routes, sort them in descending order and show top 30 routes

In [31]:
se.sql("""
SELECT -- STEP 6: COUNT the resulting routes, sort them in descending order and show top 30 routes
    route, 
    COUNT(*) AS route_count
FROM (
    SELECT  -- STEP 5: Take the FINAL TABLE to construct a route list for each `user/session` combination 
        user_id, 
        session_id, 
        concat_ws(' - ', sort_array(collect_list(struct(event_page))).event_page) as route
    FROM (
        SELECT 
            user_id,
            session_id,
            event_page
        FROM (
            SELECT 
                cl.user_id,
                cl.session_id,
                cl.event_type,
                cl.event_page,
                cl.timestamp,
                cl.timestamp_lag,
                cl.lag_event,
                err.error_event
            FROM (          -- STEP 1: CREATE A DERIVED TABLE: This table will store lagged events
                SELECT      -- We'll use the derived table to remove events ocurring recurrently on the same page
                    cl1.user_id,
                    cl1.session_id,
                    cl1.event_type,
                    cl1.event_page,
                    cl1.timestamp,
                    cl1.timestamp_lag,
                    cl2.event_page AS lag_event
                FROM (
                    SELECT
                        *, -- HERE WE OBTAIN VALUES LAGGED BY ONE POSITION FOR THE TIMESTAMP COLUMN
                        LAG(cl.timestamp, 1, 0) OVER (PARTITION BY cl.user_id, cl.session_id ORDER BY cl.timestamp ASC) AS timestamp_lag
                    FROM clicks AS cl
                ) AS cl1
                LEFT JOIN 
                    clicks AS cl2
                ON 
                    cl1.user_id = cl2.user_id
                    AND cl1.session_id = cl2.session_id
                    AND cl1.timestamp_lag = cl2.timestamp
                ORDER BY cl1.user_id, cl1.session_id, cl1.timestamp
            ) AS cl
            LEFT JOIN ( 
                SELECT user_id, session_id, timestamp, 1 AS error_event -- STEP 2: DERIVE A TABLE CONTAINING ONLY ROWS WITH USER SESSION ERRORS
                FROM clicks                                                  -- MARK ERRORS WITH 1 IN A SEPARATE COLUMN
                WHERE event_type LIKE "%error%"
            ) AS err
            ON -- STEP 3: JOIN THE TABLES FROM STEP 1 and STEP 2 SO THAT ALL ROWS WITH TIMESTAMPS ON or AFTER SESSION ERROR ARE MARKED WITH 1 AS ERRORS
                cl.user_id = err.user_id
                AND cl.session_id = err.session_id
                AND cl.timestamp >= err.timestamp
            ORDER BY cl.user_id, cl.session_id, cl.timestamp) AS tab1
        WHERE  -- STEP 4: DERIVE A FINAL TABLE BY FILTERING OUT ERRORS AND ROWS WHERE A CURRENT EVENT IS THE SAME AS THE PREVIOUS ONE
            error_event IS NULL 
            AND NOT event_page = lag_event
            OR lag_event IS NULL) AS tbl1
    GROUP BY user_id, session_id
    ORDER BY user_id, session_id) AS tbl2
GROUP BY route
ORDER BY route_count DESC;
""").show(30, False)

+-------------------------+-----------+
|route                    |route_count|
+-------------------------+-----------+
|main                     |8185       |
|archive - main           |1110       |
|main - rabota            |1045       |
|internet - main          |895        |
|bonus - main             |870        |
|main - news              |768        |
|main - tariffs           |677        |
|main - online            |587        |
|main - vklad             |517        |
|archive - main - rabota  |336        |
|bonus - main - rabota    |271        |
|main - news - rabota     |263        |
|archive - bonus - main   |260        |
|archive - internet - main|253        |
|internet - main - rabota |251        |
|archive - main - news    |238        |
|internet - main - news   |211        |
|archive - main - tariffs |206        |
|bonus - internet - main  |206        |
|internet - main - tariffs|202        |
|bonus - main - news      |169        |
|archive - main - online  |163        |
